In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

import torchvision.transforms as tr 

from dataset import make_dataset, collate_fn

In [3]:
import sys
import os

sys.path.append(os.path.abspath('..'))

from data.dataset import make_dataset, collate_fn
from torch.utils.data import DataLoader

# 설정 변수
batch_size = 4
size = 512

image_dir_train = 'train/images/'
bbox_dir_train = 'train/bbox/'

image_dir_val = 'validation/images/'
bbox_dir_val = 'validation/bbox/'

image_dir_test = 'test/images/'
bbox_dir_test = 'test/bbox/'


# 데이터셋 생성
train_set = make_dataset(
    image_dir=image_dir_train,
    bbox_dir=bbox_dir_train
)

val_set = make_dataset(
    image_dir=image_dir_val,
    bbox_dir=bbox_dir_val
)

test_set = make_dataset(
    image_dir=image_dir_test,
    bbox_dir=bbox_dir_test
)

# DataLoader 생성
train_loader = DataLoader(
    train_set, 
    batch_size=batch_size,
    shuffle=True,
    pin_memory=True,
    collate_fn=collate_fn,
    num_workers=0
)

val_loader = DataLoader(
    val_set, 
    batch_size=batch_size,
    shuffle=False,
    pin_memory=True,
    collate_fn=collate_fn,
    num_workers=0
)

test_loader = DataLoader(
    test_set, 
    batch_size=batch_size,
    shuffle=True,
    collate_fn=collate_fn
)

# 데이터 로드 및 출력
images, labels, bbox = next(iter(val_loader))

print(images.shape)
print(labels)
print(bbox)

torch.Size([4, 3, 512, 512])
(tensor([4]), tensor([4]), tensor([4]), tensor([4]))
(tensor([[ 200.9026,    0.0000, 1158.6770,  960.9201]]), tensor([[ 266.9031,  348.4540, 1536.1183,  878.4917]]), tensor([[ 897.0312,    0.0000, 1280.0000,  720.0000]]), tensor([[ 348.3173,  163.5216, 1481.7118,  965.0387]]))


# Model

In [28]:
from ultralytics import YOLO

In [29]:
model = YOLO('DL_Programming\yolov8n.pt')

<>:1: SyntaxWarning: invalid escape sequence '\y'
<>:1: SyntaxWarning: invalid escape sequence '\y'
C:\Users\User\AppData\Local\Temp\ipykernel_2644\3928075647.py:1: SyntaxWarning: invalid escape sequence '\y'
  model = YOLO('DL_Programming\yolov8n.pt')
100%|██████████| 6.23M/6.23M [00:00<00:00, 18.0MB/s]


In [30]:
print(type(model.names), len(model.names))
print(model.names)

<class 'dict'> 80
{0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 66: 'keyboard', 67: 'cell p

In [34]:
model.train(data=r'C:\Users\User\OneDrive\바탕 화면\DL\DL_Programming\data\data.yaml', epochs=10, patience=2, batch=32, imgsz=512)

Ultralytics YOLOv8.2.28  Python-3.12.2 torch-2.2.2 CPU (11th Gen Intel Core(TM) i5-1155G7 2.50GHz)
engine\trainer: task=detect, mode=train, model=DL_Programming\yolov8n.pt, data=C:\Users\User\OneDrive\ \DL\DL_Programming\data\data.yaml, epochs=10, time=None, patience=2, batch=32, imgsz=512, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False,

100%|██████████| 755k/755k [00:00<00:00, 38.9MB/s]


Overriding model.yaml nc=80 with nc=5

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\User\.netrc


Freezing layer 'model.22.dfl.conv.weight'


...

val: Scanning C:\Users\User\OneDrive\바탕 화면\DL\DL_Programming\data\validation\labels... 2406 images, 0 backgrounds, 4 corrupt: 100%|██████████| 2406/2406 [00:01<00:00, 2124.88it/s]

val: WARNING  C:\Users\User\OneDrive\ \DL\DL_Programming\data\validation\images\bundle of ropes_045_02766.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0018]
val: WARNING  C:\Users\User\OneDrive\ \DL\DL_Programming\data\validation\images\fish net_004_01515.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0004]
val: WARNING  C:\Users\User\OneDrive\ \DL\DL_Programming\data\validation\images\spring fish trap_045_04682.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0002]
val: WARNING  C:\Users\User\OneDrive\ \DL\DL_Programming\data\validation\images\tire_042_09867.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [      1.004]


val: New cache created: C:\Users\User\OneDrive\ \DL\DL_Programming\data\validation\labels.cache
Plotting labels to runs\detect\train3\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001111, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 512 train, 512 val
Using 0 dataloader workers
Logging results to runs\detect\train3
Starting training for 10 epochs...
Closing dataloader mosaic
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/10         0G      1.187      3.937      1.564         32        512:   2%|▏         | 5/263 [02:23<2:03:33, 28.73s/it]


KeyboardInterrupt: 

In [ ]:
print(type(model.names), len(model.names))
print(model.names)

In [ ]:
results = model.predict(source='data\test\images', save=True)

In [27]:
import os
import xml.etree.ElementTree as ET
from PIL import Image
from typing import Type
import torch
import torchvision.transforms as tr
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, image_dir, bbox_dir, transform):
        self.image_dir = image_dir
        self.bbox_dir = bbox_dir
        self.data_images = [file for file in os.listdir(image_dir) if file.endswith('.jpg')]
        self.transform = transform

        # 레이블 매핑 딕셔너리
        self.label_map = {
            'tire': 0,
            'spring fish trap': 1,
            'circular fish trap': 1,
            'rectangular fish trap': 1,
            'eel fish trap': 1,
            'fish net': 2,
            'wood': 3,
            'rope': 4,
            'bundle of ropes': 4
        }

    def parse_bbox_xml(self, xml_file, image_width, image_height):
        tree = ET.parse(xml_file)
        root = tree.getroot()

        boxes = []
        labels = []
        for obj in root.findall('object'):
            name = obj.find('name').text
            label = self.label_map.get(name, -1)  # 레이블 매핑, 없으면 -1
            
            if label == -1:
                continue

            bbox = obj.find('bndbox')
            xmin = float(bbox.find('xmin').text) / image_width
            ymin = float(bbox.find('ymin').text) / image_height
            xmax = float(bbox.find('xmax').text) / image_width
            ymax = float(bbox.find('ymax').text) / image_height

            # 바운딩 박스의 중심점과 너비, 높이 계산
            x_center = (xmin + xmax) / 2
            y_center = (ymin + ymax) / 2
            width = xmax - xmin
            height = ymax - ymin

            boxes.append([x_center, y_center, width, height])
            labels.append(label)

        # 객체가 1개만 있는 경우만 반환
        if len(boxes) == 1 and len(labels) == 1:
            return boxes, labels
        else:
            return None, None

    def __getitem__(self, idx):
        img_file = self.data_images[idx]
        img_path = os.path.join(self.image_dir, img_file)
        image = Image.open(img_path).convert("RGB")
        image_width, image_height = image.size

        # xml 파일 경로 찾기
        xml_file = os.path.join(self.bbox_dir, img_file.replace('.jpg', '.xml'))

        boxes, labels = self.parse_bbox_xml(xml_file, image_width, image_height)
        
        # boxes 또는 labels가 None이면 다음 데이터로 넘어감
        if boxes is None or labels is None:
            return self.__getitem__((idx + 1) % len(self.data_images))

        boxes = torch.tensor(boxes, dtype=torch.float32)
        labels = torch.tensor(labels, dtype=torch.int64)

        image = self.transform(image)

        return image, labels, boxes

    def __len__(self):
        return len(self.data_images)

def make_dataset(image_dir: str,
                 bbox_dir: str,
                 transform=tr.Compose([tr.Resize((512, 512)), 
                                       tr.ToTensor(),                                       
                                       tr.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                                       ])
                 ) -> Type[torch.utils.data.Dataset]:
    """
    Make pytorch Dataset for given task.
    Read the image using the PIL library and return it as an np.array.

    Args:
        image_dir (str): dataset directory
        bbox_dir (str) : dataset directory
        transform (torchvision.transforms) pytorch image transforms  

    Returns:
        torch.Dataset: pytorch Dataset
    """
        
    dataset = CustomDataset(image_dir=image_dir,
                            bbox_dir=bbox_dir,
                            transform=transform)
            
    return dataset

def collate_fn(batch):
    images, labels, boxes = zip(*batch)
    images = torch.stack(images, 0)
    return images, labels, boxes

# 저장할 경로
image_dir_train = 'train/images/'
bbox_dir_train = 'train/bbox/'

image_dir_val = 'validation/images/'
bbox_dir_val = 'validation/bbox/'

image_dir_test = 'test/images/'
bbox_dir_test = 'test/bbox/'

# 라벨 파일 생성 및 저장
def save_label_file(image_file, labels, boxes, output_dir):
    label_file = image_file.replace('.jpg', '.txt')
    with open(os.path.join(output_dir, label_file), 'w') as f:
        for label, box in zip(labels, boxes):
            x_center, y_center, width, height = box
            line = f"{label} {x_center} {y_center} {width} {height}\n"
            f.write(line)

def save_label_files(dataset, image_dir, bbox_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    for idx in range(len(dataset)):
        image, labels, boxes = dataset[idx]
        image_file = dataset.data_images[idx]
        save_label_file(image_file, labels, boxes, output_dir)

# train set
train_dataset = make_dataset(image_dir_train, bbox_dir_train)
save_label_files(train_dataset, image_dir_train, bbox_dir_train, 'train/labels')

# validation set
val_dataset = make_dataset(image_dir_val, bbox_dir_val)
save_label_files(val_dataset, image_dir_val, bbox_dir_val, 'validation/labels')

# test set
# test_dataset = make_dataset(image_dir_test, bbox_dir_test)
# save_label_files(test_dataset, image_dir_test, bbox_dir_test, 'test/labels')

In [26]:
import cv2
import numpy as np

def read_yolo_label(label_file):
    with open(label_file, 'r') as f:
        lines = f.readlines()
    
    labels = []
    for line in lines:
        parts = line.strip().split()
        class_id, x_center, y_center, width, height = map(float, parts)
        labels.append((int(class_id), x_center, y_center, width, height))
    
    return labels

def draw_bounding_boxes(image, labels):
    h, w, _ = image.shape
    for class_id, x_center, y_center, width, height in labels:
        x_min = int((x_center - width/2) * w)
        y_min = int((y_center - height/2) * h)
        x_max = int((x_center + width/2) * w)
        y_max = int((y_center + height/2) * h)
        
        cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
        cv2.putText(image, f'Class: {class_id}', (x_min, y_min - 10), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    
    return image

# 이미지 및 레이블 파일 경로
image_path = 'test/images/wood_048_04863.jpg'
label_file = 'test/labels/wood_048_04863.txt'

# 이미지 읽기
image = cv2.imread(image_path)
if image is None:
    print(f"이미지를 불러올 수 없습니다: {image_path}")
    exit()

# 레이블 읽기
labels = read_yolo_label(label_file)

# Bounding box 그리기
image_with_boxes = draw_bounding_boxes(image.copy(), labels)

# 결과 이미지 표시
cv2.imshow('Image with Bounding Boxes', image_with_boxes)
cv2.waitKey(0)
cv2.destroyAllWindows()